In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [34]:
!pip install keras-tcn

In [35]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization, Activation
from keras.layers import Embedding
from keras.layers import LSTM
from keras.layers.recurrent import GRU
from keras.layers.advanced_activations import PReLU
from tcn import TCN, tcn_full_summary

In [199]:
data = pd.read_csv("/content/drive/My Drive/5001/Seq/train1.csv")

In [212]:
# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=100):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset.iloc[i:(i+look_back)]['y']
        dataX.append(a)
        dataY.append(dataset.iloc[i+look_back]['y'])
    return np.array(dataX), np.array(dataY)

# fix random seed for reproducibility
np.random.seed(7)
train_x,train_y = create_dataset(data)

In [213]:
# 调整输入数据的格式
look_back=100
train_x = np.reshape(train_x, (train_x.shape[0], 1, train_x.shape[1]))

In [228]:
# create and fit the LSTM network
model = Sequential()
model.add(TCN(256, input_shape=(1, look_back),return_sequences=True))
model.add(TCN(128,return_sequences=True))
#Attention()
model.add(Dense(20))
model.add(Dense(1))
model.summary()
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(train_x, train_y, epochs=100, batch_size=32, validation_split=0.05)

Model: "sequential_24"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
tcn_22 (TCN)                 (None, 1, 256)            1521920   
_________________________________________________________________
tcn_23 (TCN)                 (None, 1, 128)            460416    
_________________________________________________________________
dense_55 (Dense)             (None, 1, 20)             2580      
_________________________________________________________________
dense_56 (Dense)             (None, 1, 1)              21        
Total params: 1,984,937
Trainable params: 1,984,937
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
258/258 [==============================] - 4s 16ms/step - loss: 33511.3008 - val_loss: 57.2662
Epoch 2/100
258/258 [==============================] - 4s 14ms/step - loss: 32.2303 - val_loss: 44.9644
Epoch 3/100
258/258 [=========

In [229]:
eva = model.evaluate(train_x,train_y)

271/271 [==============================] - 1s 5ms/step - loss: 11.4771


In [230]:
test = pd.read_csv("/content/drive/My Drive/5001/Seq/eval1.csv")
x,y = create_dataset(test)

In [231]:
for i in range(len(y)):
  xx = np.reshape(x[i,:], (1, 1, x.shape[1]))
  a = model.predict(xx)[0][0]
  if np.isnan(y[i]):
    y[i] = a
    for j in range(60):
      if i+j+1<len(y):
        x[i+j+1,60-j-1] = a
  else:
    y[i] = a

In [181]:
file = pd.DataFrame(columns=['y'],data=y)
print(file)

              y
0     38.269363
1     47.206909
2     47.903419
3     46.334892
4     47.225677
...         ...
8758  13.435734
8759  21.139618
8760  32.043030
8761  43.751236
8762  43.918472

[8763 rows x 1 columns]


In [219]:
file.to_csv('pred.csv')